# Example of DOE

<br> A good example of DOE I can think of is in medical testing. To test a new drug and measure it's effectiveness, we can control for different factors like food habits, height, weight, skin color, age group, etc to see if the drug is really effective in treating the corresponding disease.

For example we can use two factors such as age -- low age, mid age and old age
height -- low height, mid height and high height.

# Example of distributions:
    
Binomial - Suppose we buy 10 lotteries and probability of winning on each lottery is 0.10. Binomial distribution can be used to calculate the probability of winning atleast one lottery.

Geometric - Number of right swipes required on Tinder before hitting a match. :P

Poisson - I work for a small business lender as a Data Scientist. Poisson distribution can be used to model the arrival of big loan ask (in terms of dollar amount).

Exponential - In the above case, we can use exponential distribution to model the time between such loans.

Weibull - it can be used to model the wear and tear of computers used for Bitcoin mining. A computer used for bitcoin mining lasts approx 2-3 years. We can use a Weibull distribution with k > 1 to model the amount of time it takes for the computer to fail.

In [1]:
import simpy as sy
import random

num_checkers = 4
num_scanners = 4
total_time = 0
total_passengers = 1
num_passengers = 100


class Airport(object):
    def __init__(self, env, num_checkers, num_scanners):
        
        self.env = env
        self.checker = sy.Resource(env, num_checkers)
        self.scanners = []
        for i in range(0, num_scanners):
            self.scanners.append(sy.Resource(env))
                    
    
    def ID_check(self, passenger):
        ID_service_time = random.expovariate(1/0.75)
        yield self.env.timeout(ID_service_time)
    
    def scan(self, passenger):
        scan_time = random.uniform(0.5, 1)
        yield self.env.timeout(scan_time)
        
    def print_stats(self, res):
        print('%d of %d slots are allocated.' % (res.count, res.capacity))
        print('  Users:', res.users)
        print('  Queued events:', res.queue)
        
    def Passenger(self, env, number):
        global total_time #global average wait time
        global total_passengers    
        Arrivaltime = env.now
        
        with self.checker.request() as request:
            yield request
            yield env.process(self.ID_check(number))
        
        
        queue_list = []
        for scanner in self.scanners:
            queue_list.append(len(scanner.queue))
        
        min_queue_index = queue_list.index(min(queue_list))
        with self.scanners[min_queue_index].request() as request:
            yield request
            self.print_stats(self.scanners[min_queue_index])
            yield env.process(self.scan(number))
    
        pass_time = env.now - Arrivaltime
        total_time = total_time + pass_time
        total_passengers = total_passengers+1        

    def setup(self, env, num):
        arrival_int = random.expovariate(5)
        yield env.timeout(arrival_int)
        env.process(self.Passenger(env, num))
        
    
print('Airport Security')
env = sy.Environment()
ap = Airport(env, num_checkers, num_scanners)
# Start processes and run
for i in range(0,num_passengers):
    env.process(ap.setup(env, i))

env.run()

avg_time = total_time / total_passengers
print("avg_time")
print(avg_time)

Airport Security
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc4b668>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc4b5f8>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc4b978>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc4b748>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc4b6d8>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc4b710>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc4bf60>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc432b0>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc43128>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() object at 0x10cc43438>]
  Queued events: []
1 of 1 slots are allocated.
  Users: [<Request() obje

I implemented the assignment of passenger to the shortest queue for personal scanning. After running the code several times, it can be seen that using 4 scanners and 4 checkers we can keep the average wait time below 15 minutes. The time varies between 12 - 15 minutes on each run of the code.